In [1]:
from os import listdir
from os. path import join
import re
import pandas as pd

### Reddit Data

In [ ]:
path = '../data/aligned-summarization-data/eacl_sample_full/annotator_a/alignments/'
narrative_path = '../data/aligned-summarization-data/eacl_sample_full/narrative/'
files = listdir(path)

In [ ]:
refs, text = [], []
for file in files:
    with open(join(path, file), encoding='utf-8') as f:
        try:
            lines = f.readlines()
            refs.append(lines[1].replace('\n', '').replace('EXTRACTIVE: ', '').replace('[[', '').replace(']]', ''))
            # get source text
            with open(join(narrative_path, file[:-10] + 'story')) as source_f:
                source_text = ' '.join(source_f.readlines())
                text.append(source_text.replace('\n', ''))
        except:
            continue

In [ ]:
lines[1].replace('\n', '').replace('EXTRACTIVE: ', '').replace('[[', '').replace(']]', '')

In [ ]:
with open(join(narrative_path, files[0][:-10] + 'story')) as ff:
    ll = ff.readlines()

In [ ]:
data = pd.DataFrame()

In [ ]:
data['reference'] = refs
data['text'] = text

In [ ]:
data.head()

In [ ]:
data.to_csv('reddit.csv')

### CNN/ Daily News

In [2]:
cnn_path = '../data/cnn_stories/cnn/stories/'
dailymail_path = '../data/dailymail_stories/dailymail/stories/'

In [3]:
def clean_text_cnn_daily(line):
    """Clean text for CNN/Daily mail dataset."""
    line = line.lower()
    line = line.replace('\n', '').replace('(cnn)', '')
    if "@highlight" in line: return line
    if line != "":
        # fix missing period
        if line[-1] not in ['.', '?', '!']:
            return line + "."
    return line

In [22]:
files = listdir(dailymail_path)
refs, text = [], []
for file in files:
    with open(join(dailymail_path, file), encoding='utf-8') as f:
        try:
            lines = f.readlines()
            body_text = []
            reference = []
            next_is_highlight = False
            for line in lines:
                line = clean_text_cnn_daily(line)
                if line == '':  # empty
                    continue
                elif line.startswith('@highlight'):
                    next_is_highlight = True
                elif next_is_highlight:
                    reference.append(line)
                else:
                    body_text.append(line)
            
            # Make body text and reference into a single string
            text.append(' '.join(body_text))
            refs.append(' '.join(reference))
        except:
            continue

In [23]:
def save_to_dataframe(text, refs):
    data = pd.DataFrame()
    data['text'] = text
    data['reference'] = refs
    data = data[~data.duplicated()]
    data = data[data.text != '']
    return data

In [24]:
data = save_to_dataframe(text, refs)

In [25]:
data.head()

,text,reference
0,the bbc has apologised after john inverdale ma...,presenter john inverdale said 'rose-c*****' gl...
1,denver quarterback peyton manning broke brett ...,denver quarterback peyton manning reached 510 ...
2,"by . michael seamark. updated:. 05:47 est, 8 f...",gary mckinnon's family battling against his ex...
3,norwich city have taken the unexpected decisio...,neil adams appointed as norwich city's new man...
4,"becky edwards from cannock, staffs. was so hum...","becky edwards from cannock, staffs. was so hum..."


In [26]:
data.to_csv('../data/dailymail.csv', index=False)